# 第8章 文埋め込み

# 第8章 文埋め込み

## 8.4 最近傍探索ライブラリ `Faiss` を使った検索

### 8.4.2 `Faiss`を利用した最近傍探索の実装

#### 準備

In [ ]:
!pip install datasets faiss-cpu scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 72.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.7 MB/s eta 0

#### データセットの読み込みと前処理

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-paragraphsのリポジトリから
# Wikipediaの段落テキストのデータを読み込む
paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)

Generating train split:   0%|          | 0/9668476 [00:00<?, ? examples/s]

Dataset jawiki-paragraphs downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___jawiki-paragraphs/default/1.0.0/0f2d7acd99ad7ae0615fd07442dbd1654d37c5d60a39fc720efe28acff3f86f8. Subsequent calls will reuse this data.


In [ ]:
# 段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [ ]:
from pprint import pprint

# 段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [ ]:
# 段落データのうち、各記事の最初の段落のみを使うようにする
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

Filter:   0%|          | 0/9668476 [00:00<?, ? examples/s]

In [ ]:
# フィルタリング後の段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 1339236
})


In [ ]:
# フィルタリング後の段落データの内容を確認する
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '10-94194440-0',
 'pageid': 10,
 'paragraph_index': 0,
 'revid': 94194440,
 'section': '__LEAD__',
 'text': '言語(げんご)は、狭義には「声による記号の体系」をいう。',
 'title': '言語'}


#### トークナイザとモデルの準備

Hugging Face Hubから読み込む場合

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Hugging Face Hubにアップロードされた
# 教師なしSimCSEのトークナイザとエンコーダを読み込む
model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

Google ドライブに保存したモデルを読み込む場合

In [ ]:
from google.colab import drive

drive.mount("drive")

Mounted at drive


In [ ]:
!cp -r drive/MyDrive/llm-book/outputs_unsup_simcse .

In [ ]:
from transformers import AutoModel, AutoTokenizer

# ディスクに保存された教師なしSimCSEのトークナイザとエンコーダを読み込む
model_path = "outputs_unsup_simcse/encoder"
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoder = AutoModel.from_pretrained(model_path)

共通の処理

In [ ]:
# 読み込んだモデルをGPUのメモリに移動させる
device = "cuda:0"
encoder = encoder.to(device)

#### モデルによる埋め込みの計算

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]

    # ベクトルをNumPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [ ]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

Map:   0%|          | 0/1339236 [00:00<?, ? examples/s]

In [ ]:
# 埋め込みを付与した段落データの形式と事例数を確認する
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [ ]:
# 埋め込みを計算した段落データの内容を確認する
pprint(paragraph_dataset[0])

{'embeddings': [0.04253670945763588,
                -0.041921038180589676,
                -0.03232395276427269,
                0.01823267713189125,
                -0.06700421124696732,
                -0.060905277729034424,
                -0.0534023717045784,
                0.005872650071978569,
                0.005581581499427557,
                0.00042301995563320816,
                0.05438239127397537,
                -0.030172063037753105,
                -0.015410738065838814,
                -0.09762054681777954,
                0.031499966979026794,
                0.007067433558404446,
                0.004230297636240721,
                -0.018429215997457504,
                -0.07031217217445374,
                0.009732971899211407,
                0.006155171897262335,
                -0.03274840489029884,
                -0.008405999280512333,
                -0.023153288289904594,
                0.051212359219789505,
                0.04340992122888565,
        

In [ ]:
# 埋め込みを付与した段落データをディスクに保存する
paragraph_dataset.save_to_disk(
    "outputs_unsup_simcse/embedded_paragraphs"
)

Saving the dataset (0/10 shards):   0%|          | 0/1339236 [00:00<?, ? examples/s]

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

drive.mount("drive")

In [ ]:
# 保存された段落データをGoogleドライブのフォルダにコピーする
!cp -r outputs_unsup_simcse/embedded_paragraphs drive/MyDrive/llm-book/outputs_unsup_simcse

#### `Faiss` による最近傍探索を試す

In [ ]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

  0%|          | 0/1340 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})

In [ ]:
query_text = "日本語は、主に日本で話されている言語である。"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings", embed_texts([query_text])[0], k=10
)
# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(score, title, text)

0.78345203 日本の言語 日本の言語(にほんのげんご)は、日本の国土で使用されている言語について記述する。日本#言語も参照。
0.75877357 日本語教育 日本語教育(にほんごきょういく)とは、外国語としての日本語、第二言語としての日本語についての教育の総称である。
0.7494176 日本語学 日本語学(にほんごがく)とは、日本語を研究の対象とする学問である。
0.74729466 日本語 日本語(にほんご、にっぽんご、英語: Japanese)は、日本国内や、かつての日本領だった国、そして国外移民や移住者を含む日本人同士の間で使用されている言語。日本は法令によって公用語を規定していないが、法令その他の公用文は全て日本語で記述され、各種法令において日本語を用いることが規定され、学校教育においては「国語」の教科として学習を行う等、事実上、日本国内において唯一の公用語となっている。
0.7045407 国語 (教科) 国語(こくご、英: Japanese Language)は、日本の学校教育における教科の一つ。
0.7029643 和製英語 和製英語(わせいえいご)は、日本語の中で使われる和製外来語の一つで、日本で日本人により作られた、英語の言葉や英語に似ている言葉(固有名詞や商品名などを除く)である。英語圏では別表現をするために理解されなかったり、もしくは、全く異なった解釈をされたりする場合がある。
0.6956495 口語 口語(こうご)とは、普通の日常的な生活の中での会話で用いられる言葉遣いのことである。書記言語で使われる文語と違い、方言と呼ばれる地域差や社会階層などによる言語変種が応じやすく、これらと共通語などを使い分ける状態はダイグロシアと呼ばれる。
0.6944481 ジャパン ジャパン(英語: Japan)は、英語で日本を意味する単語。
0.6911353 日本語学科 日本語学科(にほんごがっか)とは、日本語を教育研究することを目的として大学や専門学校などの高等教育機関に置かれる学科の名称である。
0.6908301 日本語学校 日本語学校(にほんごがっこう)とは、主に日本語を母語としない者を対象として、第二言語・外国語としての日本語教育を実施する機関。日本国内外に存在している。


## 8.3 文埋め込みモデルの実装

### 8.3.1 教師なしSimCSEの実装

#### 準備

In [ ]:
# 必要なパッケージをインストールする
!pip install datasets scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 72.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.7 MB/s eta 0

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定する
set_seed(42)

#### データセットの読み込みと前処理

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-sentencesのリポジトリから
# Wikipediaの文のデータを読み込み、SimCSEの訓練セットとして使用する
unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

Generating train split:   0%|          | 0/24387500 [00:00<?, ? examples/s]

Dataset jawiki-sentences downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___jawiki-sentences/default/1.0.0/9ffd2ffb1fc789ae54454c9279182964ae91a31003da241385388870f8c1f3e7. Subsequent calls will reuse this data.


In [ ]:
# 訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [ ]:
# 訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

In [ ]:
# 訓練セットから空行の事例を除外する
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Filter:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [ ]:
# 訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)
# パフォーマンスの低下を防ぐため、シャッフルされた状態の訓練セットを
# ディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [ ]:
# 前処理後の訓練セットの形式と事例数を確認する
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [ ]:
# 前処理後の訓練セットの内容を確認する
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


In [ ]:
# Hugging Face Hubのllm-book/JGLUEのリポジトリから
# JSTSデータセットの訓練セットと検証セットを読み込み、
# それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="train"
)
test_dataset = load_dataset(
    "llm-book/JGLUE", name="JSTS", split="validation"
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset jglue downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___jglue/JSTS/1.1.0/afed02e914319785e72f3ea981b4bd3e00089f2361b1137820c183c6b8173edd. Subsequent calls will reuse this data.


#### トークナイザと collate 関数の準備

In [ ]:
from transformers import AutoTokenizer

# Hugging Face Hubにおけるモデル名を指定する
base_model_name = "cl-tohoku/bert-base-japanese-v3"
# モデル名からトークナイザを初期化する
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [ ]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

#### モデルの準備

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
    """SimCSEのモデル"""

    def __init__(
        self,
        base_model_name: str,
        mlp_only_train: bool = False,
        temperature: float = 0.05,
    ):
        """モデルの初期化"""
        super().__init__()

        # モデル名からエンコーダを初期化する
        self.encoder = AutoModel.from_pretrained(base_model_name)
        # パラメータをメモリ上に隣接した形で配置
        # これを実行しない場合、モデルの保存でエラーになることがある
        for param in self.encoder.parameters():
            param.data = param.data.contiguous()
        # MLP層の次元数
        self.hidden_size = self.encoder.config.hidden_size
        # MLP層の線形層
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        # MLP層の活性化関数
        self.activation = nn.Tanh()

        # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
        self.mlp_only_train = mlp_only_train
        # 交差エントロピー損失の計算時に使用する温度
        self.temperature = temperature

    def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダを用いて文をベクトルに変換"""
        # トークナイズされた文をエンコーダに入力する
        encoded_texts = self.encoder(**tokenized_texts)
        # モデルの最終層の出力（last_hidden_state）の
        # [CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # self.mlp_only_trainのフラグがTrueに設定されていて
        # かつ訓練時でない場合、MLP層の変換を適用せずにベクトルを返す
        if self.mlp_only_train and not self.training:
            return encoded_texts

        # MLP層によるベクトルの変換を行う
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.activation(encoded_texts)

        return encoded_texts

    def forward(
        self,
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels: Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        """モデルの前向き計算を定義"""
        # 文ペアをベクトルに変換する
        encoded_texts_1 = self.encode_texts(tokenized_texts_1)
        encoded_texts_2 = self.encode_texts(tokenized_texts_2)

        # 文ペアの類似度行列を作成する
        sim_matrix = F.cosine_similarity(
            encoded_texts_1.unsqueeze(1),
            encoded_texts_2.unsqueeze(0),
            dim=2,
        )

        # 交差エントロピー損失を求める
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(
            sim_matrix, positive_mask
        )

        return ModelOutput(loss=loss, scores=positive_scores)

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v3 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### `Trainer` の準備

In [ ]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """
    モデルが予測したスコアと評価用データのスコアの
    スピアマンの順位相関係数を計算
    """
    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).statistic

    return {"spearman": spearman}

In [ ]:
from transformers import TrainingArguments

# 教師なしSimCSEの訓練のハイパーパラメータを設定する
unsup_training_args = TrainingArguments(
    output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=64,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=64,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to="none",  # 外部ツールへのログを無効化
)

In [ ]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(
        self, eval_dataset: Dataset | None = None
    ) -> DataLoader:
        """
        検証・テストセットのDataLoaderでeval_collate_fnを使うように
        Trainerのget_eval_dataloaderをオーバーライド
        """
        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True,
        )

# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [ ]:
# 教師なしSimCSEの訓練を行う
unsup_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Spearman
250,0.002000,2.657105,0.690523
500,0.000400,2.568554,0.692600
750,0.000100,2.461003,0.716650
1000,0.000700,2.443940,0.722778
1250,0.000200,2.412055,0.728994
1500,0.000100,2.407356,0.730363
1750,0.000100,2.337075,0.737031
2000,0.000100,2.342816,0.736433
2250,0.000000,2.321053,0.744957
2500,0.000300,2.343930,0.737874


TrainOutput(global_step=15625, training_loss=0.0001253565512150526, metrics={'train_runtime': 5290.4647, 'train_samples_per_second': 189.019, 'train_steps_per_second': 2.953, 'total_flos': 0.0, 'train_loss': 0.0001253565512150526, 'epoch': 1.0})

#### 性能評価

In [ ]:
# 検証セットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.2473950386047363,
 'eval_spearman': 0.7622917049018967,
 'eval_runtime': 16.8507,
 'eval_samples_per_second': 738.902,
 'eval_steps_per_second': 11.572,
 'epoch': 1.0}

In [ ]:
# テストセットで教師なしSimCSEのモデルの評価を行う
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 2.1452813148498535,
 'eval_spearman': 0.8034015311347286,
 'eval_runtime': 2.0193,
 'eval_samples_per_second': 721.531,
 'eval_steps_per_second': 11.39,
 'epoch': 1.0}

#### トークナイザとモデルの保存

In [ ]:
# 教師なしSimCSEのエンコーダを保存
encoder_path = "outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('outputs_unsup_simcse/encoder/tokenizer_config.json',
 'outputs_unsup_simcse/encoder/special_tokens_map.json',
 'outputs_unsup_simcse/encoder/vocab.txt',
 'outputs_unsup_simcse/encoder/added_tokens.json')

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

# Googleドライブをマウントする
drive.mount("drive")

Mounted at drive


In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!mkdir -p drive/MyDrive/llm-book
!cp -r outputs_unsup_simcse drive/MyDrive/llm-book

### 8.3.2 教師あり SimCSE の実装

#### 準備

In [ ]:
# 乱数のシードを設定する
set_seed(42)

#### データセットの読み込みと前処理

In [ ]:
# Hugging Face Hubのllm-book/jsnliのリポジトリから
# JSNLIの訓練セットを読み込む
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

Generating train split:   0%|          | 0/533005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3916 [00:00<?, ? examples/s]

Dataset jsnli downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___jsnli/default/1.0.0/0f9249bf69fb74d71f6adee2069ada127288c4ab94cc1a91c0b0628f754893fe. Subsequent calls will reuse this data.


In [ ]:
# JSNLIの訓練セットの形式と事例数を確認する
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [ ]:
from pprint import pprint

# JSNLIの訓練セットの内容を確認する
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}


In [ ]:
import csv
import random
from typing import Iterator

# JSNLIの訓練セットから、前提文とラベルごとに仮説文をまとめたdictを作成する
premise2hypotheses = {}

premises = jsnli_dataset["premise"]  # 前提文
hypotheses = jsnli_dataset["hypothesis"]  # 仮説文
labels = jsnli_dataset["label"]  # ラベル

for premise, hypothesis, label in zip(premises, hypotheses, labels):
    if premise not in premise2hypotheses:
        premise2hypotheses[premise] = {
            "entailment": [],
            "neutral": [],
            "contradiction": [],
        }

    premise2hypotheses[premise][label].append(hypothesis)

In [ ]:
def generate_sup_train_example() -> Iterator[dict[str, str]]:
    """教師ありSimCSEの訓練セットの事例を生成"""
    # JSNLIのデータから (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文)
    # の三つ組を生成する
    for premise, hypotheses in premise2hypotheses.items():
        # 「矛盾」ラベルの仮説文が一つもない事例はスキップする
        if len(hypotheses["contradiction"]) == 0:
            continue

        # 「含意」ラベルの仮説文一つにつき、「矛盾」ラベルの仮説文一つを
        # ランダムに関連付ける
        for entailment_hypothesis in hypotheses["entailment"]:
            contradiction_hypothesis = random.choice(
                hypotheses["contradiction"]
            )
            # (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文) の三つ組を
            # dictとして生成する
            yield {
                "premise": premise,
                "entailment_hypothesis": entailment_hypothesis,
                "contradiction_hypothesis": contradiction_hypothesis,
            }

# 定義したジェネレータ関数を用いて、教師ありSimCSEの訓練セットを構築する
sup_train_dataset = Dataset.from_generator(generate_sup_train_example)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-d1c3ccc1c7d00cd2/0.0.0. Subsequent calls will reuse this data.


In [ ]:
# 訓練セットの形式と事例数を確認する
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [ ]:
# 訓練セットの内容を確認する
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


#### collate 関数の準備

In [ ]:
def sup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """訓練セットのミニバッチを作成"""
    premises = []
    hypotheses = []

    for example in examples:
        premises.append(example["premise"])

        entailment_hypothesis = example["entailment_hypothesis"]
        contradiction_hypothesis = example["contradiction_hypothesis"]

        hypotheses.extend(
            [entailment_hypothesis, contradiction_hypothesis]
        )

    # ミニバッチに含まれる前提文と仮説文にトークナイザを適用する
    tokenized_premises = tokenizer(
        premises,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )
    tokenized_hypotheses = tokenizer(
        hypotheses,
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 前提文と仮説文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2 * len(premises), 2)

    return {
        "tokenized_texts_1": tokenized_premises,
        "tokenized_texts_2": tokenized_hypotheses,
        "labels": labels,
    }

#### モデルの準備

In [ ]:
# 教師ありSimCSEのモデルを初期化する
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v3 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### `Trainer` の準備

In [ ]:
# 教師ありSimCSEの訓練のハイパーパラメータを設定する
sup_training_args = TrainingArguments(
    output_dir="outputs_sup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=128,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=128,  # 評価時のバッチサイズ
    learning_rate=5e-5,  # 学習率
    num_train_epochs=3,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
)

In [ ]:
# 教師ありSimCSEのTrainerを初期化する
sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

#### 訓練の実行

In [ ]:
# 教師ありSimCSEの訓練を行う
sup_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Spearman
250,1.435900,2.948337,0.793655
500,1.100400,2.893555,0.790319
750,1.007400,2.875876,0.789883
1000,0.976300,2.949479,0.786294
1250,0.934300,2.923010,0.793312
1500,0.829400,2.985832,0.796338
1750,0.755600,3.013074,0.794263
2000,0.759800,2.999113,0.795966
2250,0.750100,2.957278,0.799006
2500,0.730800,2.964985,0.799876


TrainOutput(global_step=4065, training_loss=0.8116521856441709, metrics={'train_runtime': 2847.1429, 'train_samples_per_second': 182.75, 'train_steps_per_second': 1.428, 'total_flos': 0.0, 'train_loss': 0.8116521856441709, 'epoch': 3.0})

#### 性能評価

In [ ]:
# 検証セットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 3.030463457107544,
 'eval_spearman': 0.8009272526316298,
 'eval_runtime': 16.6266,
 'eval_samples_per_second': 748.862,
 'eval_steps_per_second': 5.894,
 'epoch': 3.0}

In [ ]:
# テストセットで教師ありSimCSEのモデルの評価を行う
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.5370500087738037,
 'eval_spearman': 0.8158127715454032,
 'eval_runtime': 2.1569,
 'eval_samples_per_second': 675.501,
 'eval_steps_per_second': 5.563,
 'epoch': 3.0}

#### Google ドライブへの保存

In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!cp -r outputs_sup_simcse drive/MyDrive/llm-book